In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sabafathi1","key":"caf096e3f98fe2cbcbe059a2a40c9270"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle kernels output sabafathi1/VAE4cbe0d8270

Output file downloaded to /content/checkpoints/epoch_00.pth
Output file downloaded to /content/checkpoints/epoch_01.pth
Output file downloaded to /content/checkpoints/epoch_02.pth
Output file downloaded to /content/checkpoints/epoch_03.pth
Output file downloaded to /content/checkpoints/epoch_04.pth
Output file downloaded to /content/checkpoints/epoch_05.pth
Output file downloaded to /content/checkpoints/epoch_06.pth
Output file downloaded to /content/checkpoints/epoch_07.pth
Output file downloaded to /content/checkpoints/epoch_08.pth
Output file downloaded to /content/checkpoints/epoch_09.pth
Output file downloaded to /content/checkpoints/epoch_10.pth
Output file downloaded to /content/checkpoints/epoch_11.pth
Output file downloaded to /content/checkpoints/epoch_12.pth
Output file downloaded to /content/checkpoints/epoch_13.pth
Output file downloaded to /content/checkpoints/epoch_14.pth
Output file downloaded to /content/checkpoints/epoch_15.pth
Output file downloaded to /content/check

In [4]:
!kaggle kernels output sabafathi1/gan5503d4fd3c

Output file downloaded to /content/checkpoints/discriminator_epoch_0.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_1.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_10.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_11.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_12.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_13.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_14.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_15.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_16.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_17.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_18.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_19.pth
Output file downloaded to /content/checkpoints/discriminator_epoch_2.pth
Output file downloaded to /content/checkp

In [5]:
!gdown --id 1ibjbccZPik-ramrZjYDVAXILJ6sna9g3

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ibjbccZPik-ramrZjYDVAXILJ6sna9g3
From (redirected): https://drive.google.com/uc?id=1ibjbccZPik-ramrZjYDVAXILJ6sna9g3&confirm=t&uuid=43412919-4d7a-4a34-9074-66b7471a558f
To: /content/img_align_celeba.zip
100% 1.44G/1.44G [00:22<00:00, 64.4MB/s]


In [6]:
!unzip /content/img_align_celeba.zip -d /content/celeba_data > /dev/null 2>&1

In [8]:
import torch
import torch.nn as nn
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        self.lrelu = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        h = self.lrelu(self.fc1(x))
        h = self.lrelu(self.fc2(h))
        return self.fc_mean(h), self.fc_logvar(h)

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.lrelu = nn.LeakyReLU(0.2)

    def forward(self, z):
        h = self.lrelu(self.fc1(z))
        h = self.lrelu(self.fc2(h))
        x_hat = torch.sigmoid(self.fc_out(h))
        return x_hat.view(-1, 3, 64, 64)

class VAE(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std).to(device)
        return mean + eps * std

    def forward(self, x):
        mean, log_var = self.encoder(x)
        z = self.reparameterize(mean, log_var)
        x_hat = self.decoder(z)
        return x_hat, mean, log_var

In [10]:
input_dim = 3 * 64 * 64
hidden_dim = 512
latent_dim = 128

encoder = Encoder(input_dim, hidden_dim, latent_dim)
decoder = Decoder(latent_dim, hidden_dim, input_dim)
vae = VAE(encoder, decoder).to(device)

In [11]:
vae_checkpoint = torch.load("checkpoints/last_checkpoint.pth", map_location=device)
vae.load_state_dict(vae_checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
vae_checkpoint = torch.load("/kaggle/input/vae4cbe0d8270/checkpoints/last_checkpoint.pth", map_location=device)
vae.load_state_dict(vae_checkpoint['model_state_dict'])

<All keys matched successfully>

In [12]:
class Generator(nn.Module):
    def __init__(self, noise_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(noise_dim, 4 * 4 * 1024),
            nn.ReLU(True),
            nn.Unflatten(1, (1024, 4, 4)),
            nn.ConvTranspose2d(1024, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 3, 4, 2, 1),
            nn.Tanh()
        )
    def forward(self, x):
        return self.main(x)

In [13]:
noise_dim = 100

gan = Generator(noise_dim).to(device)

In [15]:
gan_checkpoint = torch.load("checkpoints/generator_last_checkpoint.pth", map_location=device)
gan.load_state_dict(gan_checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
gan_checkpoint = torch.load("/kaggle/input/gan5503d4fd3c/checkpoints/generator_last_checkpoint.pth", map_location=device)
gan.load_state_dict(gan_checkpoint['model_state_dict'])

<All keys matched successfully>

In [16]:
vae.eval()
with torch.no_grad():
    z = torch.randn(12000, latent_dim).to(device)
    vae_samples = decoder(z)

In [17]:
import torchvision.utils as vutils
from PIL import Image
import os

os.makedirs("vae_images", exist_ok=True)

vae_samples = vae_samples.cpu()
for i, img_tensor in enumerate(vae_samples):
    img = img_tensor.clamp(0, 1)
    img_pil = vutils.make_grid(img, normalize=False).permute(1, 2, 0).numpy() * 255
    img_pil = Image.fromarray(img_pil.astype("uint8"))
    img_pil.save(f"vae_images/img_{i:04d}.png")

In [18]:
gan.eval()

with torch.no_grad():
    z = torch.randn(12000, noise_dim).to(device)
    gan_samples = gan(z).cpu()

In [19]:
from PIL import Image
import os

os.makedirs("gan_images", exist_ok=True)

gan_samples = gan_samples.cpu()
for i, img_tensor in enumerate(gan_samples):
    img = img_tensor.clamp(0, 1)
    img_pil = vutils.make_grid(img, normalize=False).permute(1, 2, 0).numpy() * 255
    img_pil = Image.fromarray(img_pil.astype("uint8"))
    img_pil.save(f"gan_images/img_{i:04d}.png")

In [20]:
import os
import shutil
import random
from pathlib import Path

celeba_dir = Path("/content/celeba_data/img_align_celeba")
output_dir = Path("./real_images")
output_dir.mkdir(parents=True, exist_ok=True)
all_images = list(celeba_dir.glob("*.jpg"))

selected_images = random.sample(all_images, 12000)
for img_path in selected_images:
    shutil.copy(img_path, output_dir / img_path.name)

In [22]:
!pip install torch_fidelity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 745.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [23]:
import torch_fidelity
print(torch_fidelity.__version__)


0.3.0


In [26]:
from torch_fidelity import calculate_metrics

metrics1 = calculate_metrics(
    input1='gan_images',
    input2='real_images',
    cuda=True,
    isc=True,
    fid=True,
    kid=True,
    prc=True,
    verbose=False)

for i in metrics1:
    print(i,metrics1[i])

inception_score_mean 2.144097185793562
inception_score_std 0.05696837279206358
frechet_inception_distance 93.0731547767057
kernel_inception_distance_mean 0.1072647786140442
kernel_inception_distance_std 0.0016816425411287856
recall 0.21105456923145681
precision 0.17862311004513571

In [27]:
metrics2 = calculate_metrics(
    input1='vae_images',
    input2='real_images',
    cuda=True,
    isc=True,
    fid=True,
    kid=True,
    prc=True,
    verbose=False)


for i in metrics2:
    print(i,metrics2[i])

inception_score_mean 1.6551587103698189
inception_score_std 0.015152151895093915
frechet_inception_distance 153.0719268117519
kernel_inception_distance_mean 0.17037234783172608
kernel_inception_distance_std 0.001894801673483931
recall 0.68205269932148312
precision 0.0053215986301470

In [ ]:
metrics2 = calculate_metrics(
    input1='/kaggle/working/vae_images',
    input2='/kaggle/working/real_images',
    cuda=True,
    isc=True,
    fid=True,
    kid=True,
    prc=True,
    verbose=False)


for i in metrics2:
    print(i,metrics2[i])